# California ISO API

All API code below from: Manu Kalia, https://github.com/manukalia/caiso_day-ahead_price_fetch/blob/master/utility_caiso_da_price_fetch.py

API calls to California ISO http://oasis.caiso.com/mrioasis/logon.do

Hourly electricity prices (Day Ahead Market) from BAYSHOR2_1_N001 ISO (Independent System Operator) in San Fransisco, CA from March 26, 2017 to June 23, 2020

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import pyplot
import seaborn as sns

In [7]:
import wget, os, zipfile, glob, shutil
import time, pickle, re
import PySimpleGUI as sg

In [25]:
#    FUNCTION DEFINITIONS


# Function to generate a datetime range ...
#   parameters:  datetime range start (str), end (str),
#                and frequency (defaults to 'H')

def gen_daterange(query_start, query_end, query_freq='H'):
    return pd.date_range(start = query_start,
                         end   = query_end,
                         freq  = query_freq)


# Function to determine number of batches and the size of the last batch
#   parameters:  date index, batch size (no. elements per batch), k (for-loop index)

def create_batches(date_index, batch_size):
    if len(date_index) % batch_size == 0:
        num_batches = len(date_index) // batch_size
        final_batch_size = batch_size
    else:
        num_batches = len(date_index) // batch_size + 1
        final_batch_size = len(date_index) % batch_size

    return num_batches, final_batch_size



# Function to determine number of batches and the size of the last batch
#   parameters:  date index, batch size (no. elements per batch), final batch size,
#                k (for-loop index)

def gen_batch_start_end(date_index, batch_size, final_batch_size, k):
    start_datetime = date_index[k * batch_size]

    if i == num_batches - 1:
        end_datetime   = date_index[(k * batch_size) + final_batch_size - 1]
    else:
        end_datetime   = date_index[(k+1) * batch_size - 1]

    start_arg = f'{start_datetime.year}'\
+f'{start_datetime.month}'.zfill(2)\
+f'{start_datetime.day}'.zfill(2)+'T'\
+f'{start_datetime.hour}'.zfill(2)+':00-0000'

    end_arg = f'{end_datetime.year}'\
+f'{end_datetime.month}'.zfill(2)\
+f'{end_datetime.day}'.zfill(2)+'T'\
+f'{end_datetime.hour}'.zfill(2)+':00-0000'

    return start_arg, end_arg


# Function to construct a GENERIC one-month .csv LMP price download
#    data query for the CAISO OASIS API

# Parameters (all strings):
#    node                       name of node
#

def gen_price_query(queryname, version, start_arg, end_arg, market_run_id, node):
    oasis_website = 'oasis.caiso.com'
    context_path  = 'oasisapi'
    url = f'http://{oasis_website}/{context_path}/SingleZip'
    resultformat  =  '6'

    query = f'{url}?resultformat={resultformat}&queryname={queryname}&version={version}\
&startdatetime={start_arg}&enddatetime={end_arg}\
&market_run_id={market_run_id}&node={node}'

    return query


# Function unzips all files in a specified targetdirectory,
# saving unzipped files to a specified destination directory
# Parameters:  download (target dir), unzipped destination dir

def unzip_dir(download_dir, unzipped_dest_dir):
    for item in os.listdir(download_dir):           # loop through items in dir
        if item.split('.')[-1] == 'zip':            # check for zip extension
            file_name = f'{download_dir}{item}'     # get relative path of files
            print(f'unzipping... {file_name}')
            zip_ref = zipfile.ZipFile(file_name)    # create zipfile object
            with zip_ref as destination:
                destination.extractall(unzipped_dest_dir)
        else: continue




#  USER INPUTS WINDOW LOOP


sg.ChangeLookAndFeel('GreenMono')

input_window_layout = [
    [sg.Text('\nEnter a CAISO nodename  (or comma-separated list of\n\
nodenames) below.  Then hit the SUBMIT button to proceed ...',
        font=('Raleway', 18))],
    [sg.InputText('BAYSHOR2_1_N001, LCIENEGA_6_N001, SOUTHBY_6_N001',
        key='nodename_string',
        font=("Raleway", 14),
        size=(55, 1))],

    [sg.Text('\n\n', font=("Raleway", 6))],
    [sg.Text('Enter the start & end datetimes for the hourly price downloads:',
        font=("Raleway", 18))],
    [sg.InputText('2019-01-01 00:00:00',
        key='start_date',
        font=("Raleway", 14),
        size=(20, 1)), sg.CalendarButton('select start date', target='start_date')],
    [sg.InputText('2019-12-01 00:00:00',
        key='end_date',
        font=("Raleway", 14),
        size=(20, 1)), sg.CalendarButton('select end  date', target='end_date')],

    [sg.Text('\n\n', font=("Raleway", 6))],
    [sg.Frame(layout=[
        [sg.Checkbox('Keep downloaded .zip files',
            size=(25,1),
            font=("Raleway", 14),
            default=False,
            key='keep_zipped_files'),
        sg.Checkbox('Keep unzipped .csv files',
            size=(25,1),
            font=("Raleway", 14),
            key='keep_unzipped_files',
            default=False)],
        [sg.Checkbox('Save output as .csv',
            size=(25,1),
            font=("Raleway", 14),
            key='save_csv_files',
            default=True)],
        [sg.Checkbox('Save output as binarized Pandas dataframe (.pkl)',
            size=(40,1),
            font=("Raleway", 14),
            key='save_dataframe',
            default=True)]
            ],
        title='FILE SAVE OPTIONS',
        font=("Raleway", 18),
        title_color='darkblue',
        relief=sg.RELIEF_SUNKEN)
        ],

    [sg.Text('\n', font=("Raleway", 6))],
    [sg.Text('_' * 80)],
    [sg.Text('Choose filesave location:', font=("Raleway", 14))],
    [sg.InputText('/Users/owner',
        key='destination_folder',
        size=(55, 1),
        font=("Raleway", 14)),
        sg.FolderBrowse(target='destination_folder')],
    [sg.Submit(), sg.Cancel()]]

input_window = sg.Window('CAISO Day-ahead Prices Fetch Utility',
    input_window_layout,
    location=(180, 120),
    default_element_size=(20, 1),
    grab_anywhere=False)

event, user_input_dict = input_window.read()
input_window.close()


sg.PopupScrolled('\n\nwindow auto closes in 15 sec   (or hit OK to close)\n\n',
                 'You entered the following parameters:\n',
                 f'Node names:  {user_input_dict["nodename_string"]}',
                 f'Start date:     {user_input_dict["start_date"]}',
                 f'End date:      {user_input_dict["end_date"]}',
                 f'Keep zipped files:     {user_input_dict["keep_zipped_files"]}',
                 f'Keep unzipped files:    {user_input_dict["keep_unzipped_files"]}',
                 f'Save output .csv files:  {user_input_dict["save_csv_files"]}',
                 f'Save output dataframes:  {user_input_dict["save_dataframe"]}',
                 f'Destination folder:  {user_input_dict["destination_folder"]}',
                 font=('Raleway', 18),
                 title='',
                 size=(80, 20),
                 location=(180, 120),
                 auto_close=True,
                 auto_close_duration=15)


with open(f"{user_input_dict['destination_folder']}/user_input_dict.pkl", 'wb') as f:
    pickle.dump(user_input_dict, f)


# Parameters: API-query-starting datetime, ending datetime, and frequency,
#               batch size (no. of elements per batch), node (id str), delay (sec))

nodename_list = re.split('\W+', user_input_dict['nodename_string'])

query_start = user_input_dict['start_date']
query_end   = user_input_dict['end_date']

batch_size  = 24*28               # 28 days is safely less than 31 day API restriction
delay = 5                         # No. seconds to delay between API queries


# Call the appropriate functions and store returned results into variable names

date_index = gen_daterange(query_start, query_end)
num_batches, final_batch_size = create_batches(date_index, batch_size)


# Create temporary working directories for downloads and unzipped files

pathname = user_input_dict['destination_folder'] + '/temp_dir/'
download_pathname = pathname + 'downloads/'
unzipped_pathname = pathname + 'unzipped/'

os.mkdir(pathname)
os.mkdir(download_pathname)
os.mkdir(unzipped_pathname)



# Progress Bar and Data Fetch Loop

num_iterations = num_batches * len(nodename_list)

# Progress Bar Layout

prog_bar_layout = [[sg.Text('CAISO DA Price Download Progress',
                            font=('Raleway', 18))],
                   [sg.ProgressBar(num_iterations,
                                   orientation='h',
                                   size=(60, 12),
                                   key='progbar')],
                   [sg.Cancel()]]


# Create Progress Bar Window

prog_bar_window = sg.Window('CAISO DA Prices Download Progress',
    prog_bar_layout,
    location=(180, 120))

counter = 0

for node in nodename_list:
    for i in range(num_batches):
        event, values = prog_bar_window.read(timeout=100)
        if event == 'Cancel' or event is None: break
        start_arg, end_arg = gen_batch_start_end(date_index,
                                                 batch_size,
                                                 final_batch_size,
                                                 i)
        wget.download(gen_price_query('PRC_LMP',
                                  '1',
                                  start_arg,
                                  end_arg,
                                  'DAM',
                                  node),
                      download_pathname)
        counter += 1
        prog_bar_window['progbar'].update_bar(counter)
        time.sleep(delay)

time.sleep(4)             # wait 4 sec before closing prog bar window
prog_bar_window.close()

unzip_dir(download_pathname, unzipped_pathname)



# Create single combined dataframe of all downloaded data

dam_orig_cols=['INTERVALSTARTTIME_GMT',
               'NODE',
               'MARKET_RUN_ID',
               'XML_DATA_ITEM',
               'MW']

dam_new_cols =['datetime',
               'node',
               'market',
               'price_component',
               'dam_price_per_mwh']

dam_rename_dict = {old: new for old, new in zip(dam_orig_cols, dam_new_cols)}

dam_df = pd.DataFrame(columns=dam_new_cols)

for file in glob.glob(unzipped_pathname +'*.csv'):
    df = pd.read_csv(file, usecols=dam_orig_cols).rename(index=str,
                                                         columns=dam_rename_dict)
    df = df[df.price_component == 'LMP_PRC']
    dam_df = dam_df.append(df, ignore_index=True)

dam_df = dam_df.sort_values(by='datetime').reset_index(drop=True)

dam_df['datetime'] = pd.to_datetime(dam_df['datetime'], utc=True)
dam_df.set_index('datetime', inplace=True)
dam_df.sort_index(inplace=True)

dam_df.index = dam_df.index.tz_convert('US/Pacific')     # not sure if this is the right adjustment



# Split Combined Datatrame into Separate Node-specific Dataframes

new_nodenames = [node[:node.find('_')] if node.find('_') != (-1) else node for node in nodename_list]

df_list = []
df_names_list = []
da_price_col_names = []

for index, node in enumerate(nodename_list):
    df_list.append(new_nodenames[index] + '_df')
    df_names_list.append(new_nodenames[index] + '_df')
    da_price_col_names.append(new_nodenames[index] + '_da_price_per_mwh')

for i in range(len(new_nodenames)):
    temp_dict = dam_df[dam_df.node == nodename_list[i]].to_dict('index')
    df_list[i] = pd.DataFrame.from_dict(temp_dict, orient='index')
    df_list[i].rename(columns={'dam_price_per_mwh': da_price_col_names[i]},
                      inplace=True)
    df_list[i].drop(columns=['node'], inplace=True)



# Recombine Nodal DF's into a Single Datafram with Side-by-side Price Columns

if len(df_names_list) <= 1: pass

elif len(df_names_list) == 2:
    CAISO_DA_PRICES_mult_cols_df = df_list[0].join(df_list[1][da_price_col_names[1]],
                                          how='outer')
    df_names_list.append('CAISO_DA_PRICES_mult_cols_df')
    df_list.append(CAISO_DA_PRICES_mult_cols_df)

else:
    CAISO_DA_PRICES_mult_cols_df = df_list[0].join(df_list[1][da_price_col_names[1]],
                                          how='outer')
    for i in range(2, len(df_list)):
        CAISO_DA_PRICES_mult_cols_df = CAISO_DA_PRICES_mult_cols_df.join(df_list[i][da_price_col_names[i]],
                                                       how='outer')
    df_names_list.append('CAISO_DA_PRICES_mult_cols_df')
    df_list.append(CAISO_DA_PRICES_mult_cols_df)



# HOUSEKEEPING ...
#    -  Save dataframes as .pkl binary files
#    -  Save dataframes to .csv files
#    -  If specified by user, delete temp directory, downloaded .zip files, & unzipped .csv files

orig_path = pathname.replace('temp_dir/', '')

for index, df in enumerate(df_list):
    if user_input_dict['save_dataframe']:
        with open(orig_path + df_names_list[index] + '.pkl', 'wb') as f:
            pickle.dump(df, f)

    if user_input_dict['save_csv_files']:
        df.to_csv(orig_path + df_names_list[index] + '.csv')

if (user_input_dict['keep_zipped_files'] == False) and (user_input_dict['keep_unzipped_files'] == False):
    shutil.rmtree(pathname, ignore_errors=True)

elif user_input_dict['keep_zipped_files'] == False:
    shutil.rmtree(download_pathname, ignore_errors=True)

elif user_input_dict['keep_unzipped_files'] == False:
    shutil.rmtree(unzipped_pathname, ignore_errors=True)



# Program Complete Message

sg.PopupScrolled('\n\nwindow auto closes in 15 sec   (or hit OK to close)\n\n',
                 'Program is Complete!  Check your destination directory for saved files:\n',
                 f'Destination folder:  {user_input_dict["destination_folder"]}',
                 font=('Raleway', 18),
                 title='',
                 size=(40, 10),
                 location=(180, 120),
                 auto_close=True,
                 auto_close_duration=10)

unzipping... /Users/frederickdenbleyker/Documents/temp_dir/downloads/20200120_20200217_PRC_LMP_DAM_20200628_08_46_22_v1.zip
unzipping... /Users/frederickdenbleyker/Documents/temp_dir/downloads/20200511_20200608_PRC_LMP_DAM_20200628_08_47_23_v1.zip
unzipping... /Users/frederickdenbleyker/Documents/temp_dir/downloads/20200608_20200626_PRC_LMP_DAM_20200628_08_47_38_v1.zip
unzipping... /Users/frederickdenbleyker/Documents/temp_dir/downloads/20200413_20200511_PRC_LMP_DAM_20200628_08_47_08_v1.zip
unzipping... /Users/frederickdenbleyker/Documents/temp_dir/downloads/20200316_20200413_PRC_LMP_DAM_20200628_08_46_52_v1.zip
unzipping... /Users/frederickdenbleyker/Documents/temp_dir/downloads/20200217_20200316_PRC_LMP_DAM_20200628_08_46_36_v1.zip


'__TIMEOUT__'

# DarkSky API

## 1st wrapper, I couldn't get to work

In [5]:
from darksky.api import DarkSky, DarkSkyAsync                # by Detrous, https://github.com/Detrous/darksky
from darksky.types import languages, units, weather
from datetime import datetime as dt

In [29]:
API_KEY = 'xxxxxxxxxxxxxxxxxxxxxxxxxxx'   #historic

darksky = DarkSky(API_KEY)
t = dt(2019, 3, 26, 12)

latitude = 37.746088
longitude = -122.403058
forecast = darksky.get_time_machine_forecast(
    latitude, longitude,
    extend=False, # default `False`
    lang=languages.ENGLISH, # default `ENGLISH`
    values_units=units.AUTO, # default `auto`
    exclude=[weather.MINUTELY, weather.ALERTS], # default `[]`,
    timezone=None , # default None - will be set by DarkSky API automatically
    time=t
)

NameError: name 'DarkSky' is not defined

In [15]:
forecast.latitude # 42.3601
forecast.longitude # -71.0589
forecast.timezone # timezone for coordinates. For exmaple: `America/New_York`

forecast.currently # CurrentlyForecast. Can be found at darksky/forecast.py
forecast.minutely # MinutelyForecast. Can be found at darksky/forecast.py
forecast.hourly # HourlyForecast. Can be found at darksky/forecast.py
forecast.daily # DailyForecast. Can be found at darksky/forecast.py
forecast.alerts # [Alert]. Can be found at darksky/forecast.py

[]

In [80]:
from datetime import datetime
from typing import List

from . import base

class CurrentlyForecast(base.AutoInit):
    time: datetime
    summary: str = None
    icon: str
    nearest_storm_distance: int
    nearest_storm_bearing: int
    precip_intensity: float
    precip_intensity_error: float
    precip_probability: float
    precip_type: str
    precipAccumulation: float
    temperature: float
    apparent_temperature: float
    dew_point: float
    humidity: float
    pressure: float
    wind_speed: float
    wind_gust: float
    wind_bearing: int
    cloud_cover: float
    uv_index: int
    visibility: float
    ozone: float


class MinutelyForecastItem(base.AutoInit):
    time: datetime
    precip_intensity: float
    precip_intensity_error: float
    precip_probability: float
    precip_type: str


class MinutelyForecast(base.BaseWeather):
    data: List[MinutelyForecastItem]
    data_class = MinutelyForecastItem


class HourlyForecastItem(base.AutoInit):
    time: datetime
    summary: str = None
    icon: str
    precip_intensity: float
    precip_probability: float
    precip_type: str
    precipAccumulation: float
    temperature: float
    apparent_temperature: float
    dew_point: float
    humidity: float
    pressure: float
    wind_speed: float
    wind_gust: float
    wind_bearing: int
    cloud_cover: float
    uv_index: int
    visibility: float
    ozone: float


class HourlyForecast(base.BaseWeather):
    data: List[HourlyForecastItem]
    data_class = HourlyForecastItem


class DailyForecastItem(base.AutoInit):
    time: datetime
    summary: str = None
    icon: str
    sunrise_time: datetime
    sunset_time: datetime
    moon_phase: float
    precip_intensity: float
    precip_intensity_max: float
    precip_intensity_max_time: datetime
    precip_probability: float
    precip_type: str
    precipAccumulation: float
    temperature_high: float
    temperature_high_time: datetime
    temperature_low: float
    temperature_low_time: datetime
    apparent_temperature_high: float
    apparent_temperature_high_time: datetime
    apparent_temperature_low: float
    apparent_temperature_low_time: datetime
    dew_point: float
    humidity: float
    pressure: float
    wind_speed: float
    wind_gust: float
    wind_gust_time: datetime
    wind_bearing: int
    cloud_cover: float
    uv_index: int
    uv_index_time: datetime
    visibility: int
    ozone: float
    temperature_min: float
    temperature_min_time: datetime
    temperature_max: float
    temperature_max_time: datetime
    apparent_temperature_min: float
    apparent_temperature_min_time: datetime
    apparent_temperature_max: float
    apparent_temperature_max_time: datetime


class DailyForecast(base.BaseWeather):
    data: List[DailyForecastItem]
    data_class = DailyForecastItem


class Alert(base.AutoInit):
    title: str
    regions: list
    severity: str
    time: datetime
    expires: datetime
    description: str
    uri: str


class Flags(base.AutoInit):
    sources: List[str]
    sources_class = str
    nearest__station: float
    darksky__unavailable: bool
    units: str


class Forecast:
    latitude: float
    longitude: float
    timezone: str
    currently: CurrentlyForecast
    minutely: MinutelyForecast
    hourly: HourlyForecast
    daily: DailyForecast
    alerts: List[Alert]
    flags: Flags
    offset: int

    def __init__(
        self,
        latitude: float,
        longitude: float,
        timezone: str,
        currently: dict = None,
        minutely: dict = None,
        hourly: dict = None,
        daily: dict = None,
        alerts: [dict] = None,
        flags: dict = None,
        offset: int = None,
    ):
        self.latitude = latitude
        self.longitude = longitude
        self.timezone = timezone

        self.currently = CurrentlyForecast(
            timezone=timezone, **(currently or {}))
        self.minutely = MinutelyForecast(timezone=timezone, **(minutely or {}))
        self.hourly = HourlyForecast(timezone=timezone, **(hourly or {}))
        self.daily = DailyForecast(timezone=timezone, **(daily or {}))

        self.alerts = [Alert(timezone=timezone, **alert)
                       for alert in (alerts or [])]
        self.flags = Flags(timezone=timezone, **(flags or {}))

        self.offset = offset

ImportError: attempted relative import with no known parent package

## 2nd wrapper, I was able to get the forecast, but not historical data

In [57]:
from forecastiopy import *         # by Angel Hernandez III, https://github.com/bitpixdigital/forecastiopy3

In [58]:
apikey = '8acee58202119aceebe1a3bdd933d455'

SanFran = [37.746088, -122.403058]

fio = ForecastIO.ForecastIO(apikey,
                            units=ForecastIO.ForecastIO.UNITS_SI,
                            lang=ForecastIO.ForecastIO.LANG_ENGLISH,
                            latitude=SanFran[0], longitude=SanFran[1])

print('Latitude', fio.latitude, 'Longitude', fio.longitude)
print('Timezone', fio.timezone, 'Offset', fio.offset)
print(fio.get_url()) # You might want to see the request url

Latitude 37.746088 Longitude -122.403058
Timezone America/Los_Angeles Offset -7
https://api.darksky.net/forecast/8acee58202119aceebe1a3bdd933d455/37.746088,-122.403058?units=si&lang=en


In [59]:
if fio.has_hourly() is True:
	hourly = FIOHourly.FIOHourly(fio)
	print('Hourly')
	print('Summary:', hourly.summary)
	print('Icon:', hourly.icon)

	for hour in range(0, hourly.hours()):
		print('Hour', hour+1)
		for item in hourly.get_hour(hour).keys():
			print(item + ' : ' + str(hourly.get_hour(hour)[item]))
		# Or access attributes directly for a given minute.
		# hourly.hour_5_time would also work
		print(hourly.hour_3_time)
else:
	print('No Hourly data')

Hourly
Summary: Partly cloudy throughout the day.
Icon: partly-cloudy-day
Hour 1
time : 1593273600
summary : Possible Drizzle
icon : rain
precipIntensity : 0.0557
precipProbability : 0.34
precipType : rain
temperature : 13.2
apparentTemperature : 13.2
dewPoint : 12.38
humidity : 0.95
pressure : 1011.3
windSpeed : 3.66
windGust : 5.33
windBearing : 239
cloudCover : 0.62
uvIndex : 3
visibility : 14.88
ozone : 301.7
1593284400
Hour 2
time : 1593277200
summary : Partly Cloudy
icon : partly-cloudy-day
precipIntensity : 0.0716
precipProbability : 0.11
precipType : rain
temperature : 14.08
apparentTemperature : 14.08
dewPoint : 12.59
humidity : 0.91
pressure : 1011.4
windSpeed : 4.21
windGust : 5.86
windBearing : 247
cloudCover : 0.59
uvIndex : 4
visibility : 14.734
ozone : 301.2
1593284400
Hour 3
time : 1593280800
summary : Partly Cloudy
icon : partly-cloudy-day
precipIntensity : 0
precipProbability : 0
temperature : 14.96
apparentTemperature : 14.96
dewPoint : 12.86
humidity : 0.87
pressure

## 3rd wrapper, I couldn't get to work

In [1]:
from darksky import forecast     #by Lukáš Kubiš,  https://github.com/lukaskubis/darkskylib

In [26]:
key = 'xxxxxxxxxxxxxxxxxxxxx'

In [27]:
SanFran = forecast(key, 37.746088, -122.403058)

TypeError: 'module' object is not callable

In [28]:
SANFRAN = key, 37.746088, -122.403058
from datetime import datetime as dt
t = dt(2017, 3, 25, 12).isoformat()
sanfran = forecast(*SANFRAN, time=t)
sanfran.time

TypeError: 'module' object is not callable

## I ended up paying to download historical weather data from visualcrossing.com. This was done due to the tight deadline, but can use the API for visualcrossing.com in the near future.